# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [4]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import numpy as np
import requests
import json

# Import API key
from api_keys import geoapify_key

In [7]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()
city_data_df.count()
clean_city_data_df = city_data_df.dropna(how='any')
clean_city_data_df.count()

City_ID       419
City          419
Lat           419
Lng           419
Max Temp      419
Humidity      419
Cloudiness    419
Wind Speed    419
Country       419
Date          419
dtype: int64

---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [8]:
#%%capture --no-display

# Configure the map plot
city_map = clean_city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 600,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City"
)
# Save the figure
plt.savefig("output_data/Fig13.png")

# Display the map
city_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

In [12]:
# Narrow down cities that fit criteria and drop any results with null values
myidealtemp_df = clean_city_data_df[clean_city_data_df["Max Temp"] >= 24]

myidealtemp_df.head()
print(myidealtemp_df.count())

City_ID       138
City          138
Lat           138
Lng           138
Max Temp      138
Humidity      138
Cloudiness    138
Wind Speed    138
Country       138
Date          138
dtype: int64


### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [24]:
# Narrow down cities that fit criteria and drop any results with null values
myidealtemp_df = clean_city_data_df[clean_city_data_df["Max Temp"] >= 24]


# Drop any rows with null values
myidealtemp_clean_df = myidealtemp_df.dropna(axis=1)

# Display sample data
myidealtemp_clean_df.head()
print(myidealtemp_clean_df.count())

City_ID       138
City          138
Lat           138
Lng           138
Max Temp      138
Humidity      138
Cloudiness    138
Wind Speed    138
Country       138
Date          138
dtype: int64


### Step 3: Create a new DataFrame called `hotel_df`.

In [25]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = myidealtemp_clean_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ''

# Display sample data
hotel_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,0,shalkar,48.3119,48.6945,28.13,25,86,5.08,KZ,1691884505,
3,3,dazhong,33.0296,120.4355,28.84,78,0,2.25,CN,1691884508,
4,4,dondaicha,21.3074,74.7034,25.06,80,97,5.98,IN,1691884508,
5,5,kidal,21.0200,-0.1768,34.44,22,5,4.14,DZ,1691884509,
6,6,albu kamal,34.1218,40.8167,34.01,21,7,3.04,SY,1691884510,


In [26]:
# latitude = hotel_df["Lat"]
# print(latitude)
# longitude = hotel_df["Lng"]
# print(longitude)

# params["filter"] = f"circle:{longitude},{latitude},{radius}"
# params["bias"] = f"proximity:{longitude},{latitude}"

# params

### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [38]:
##Zoom = API and Geospatial Data Visual... @ 1:13:23 the Nearest Restaurants in Madrid activity

# Set parameters to search for a hotel
radius = 50000
params = {
    "apiKey": geoapify_key,
    "categories": "accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
#     print(latitude)
    longitude =row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    #print(params)
    ## Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionary
    name_address = requests.get(base_url,params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    #print(name_address)
    
    ## Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    ## Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

## Display sample data
hotel_df
print(hotel_df.count())

Starting hotel search
shalkar - nearest hotel: No hotel found
dazhong - nearest hotel: No hotel found
dondaicha - nearest hotel: No hotel found
kidal - nearest hotel: No hotel found
albu kamal - nearest hotel: No hotel found
eregli - nearest hotel: No hotel found
al muwayh - nearest hotel: فندق ومطعم الموية
kerema - nearest hotel: No hotel found
qurayyat - nearest hotel: No hotel found
aegina - nearest hotel: Bastione Malvasia Hotel
washington court house - nearest hotel: Holiday Inn Express
pahrump - nearest hotel: Armagosa Hotel
anantapur - nearest hotel: No hotel found
kayenta - nearest hotel: No hotel found
dame-marie - nearest hotel: No hotel found
douglas - nearest hotel: Hampton Inn & Suites Douglas
tabriz - nearest hotel: No hotel found
hakui - nearest hotel: No hotel found
ubari - nearest hotel: بيت الشباب لفجيج
parintins - nearest hotel: No hotel found
aberdeen - nearest hotel: No hotel found
turabah - nearest hotel: No hotel found
farshut - nearest hotel: No hotel found
ahra

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [54]:
#%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 600,
    frame_height = 500,
    size = "Max Temp",
    scale = 0.5,
    color = "Hotel Name",
    hover_cols = ['Hotel Name', 'Country']
)

# Save the figure
plt.savefig("output_data/Fig14.png")

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Hotel Name,Max Temp,Country)